<a href="https://colab.research.google.com/github/beallac/iitsec2023/blob/main/IITSEC_2023_Experiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Analysis - Single
=================

In [ ]:
import pandas as pd

# Load the fNIR data
data_path   = 'https://raw.githubusercontent.com/beallac/iitsec2023/main/alex.oxy'
marker_path = 'https://raw.githubusercontent.com/beallac/iitsec2023/main/alex.mrk'

# Alertnatively, you can use google's transient disk storage for "local" file access
# data_path   = 'alex.oxy'
# marker_path = 'alex.mrk'

# Generate column names as per the specified pattern
column_names = ['time']
for i in range(1, 19):
    column_names.extend([f'opt_{i}_deoxy', f'opt_{i}_oxy'])
column_names.append('counter')

# Load the data, skipping the first 8 rows, using tab as the delimiter, and assigning the column names
data = pd.read_csv(data_path, delimiter='\t', header=None, skiprows=8, names=column_names)

marker = pd.read_csv(marker_path, delimiter='\t', header=None, skiprows=6, names=['time_stamp', 'marker_code', 'unused_code'])


In [ ]:
# Process marker data to find start and stop times for each condition
# Assuming consecutive rows with the same marker indicate a continuous condition
conditions = {}
for index, row in marker.iterrows():
    marker = row['marker_code']
    if marker in conditions:
        # Assuming markers always come in pairs, we append the stop time
        conditions[marker].append(row['time_stamp'])
    else:
        # Start of a new marker condition
        conditions[marker] = [row['time_stamp']]

# Ensure each marker has a pair, if not, assign 'inf' as the stop time
for marker, times in conditions.items():
    if len(times) % 2 != 0:
        times.append(float('inf'))

# Add a condition index to the main data frame
def get_condition(time):
    for marker, times in conditions.items():
        for i in range(0, len(times), 2):
            if times[i] <= time < times[i + 1]:
                return marker
    return None
data.insert(1, 'condition', data['time'].apply(get_condition))

In [ ]:
import numpy as np
from scipy.signal import lfilter

# Assuming 'data' is your main DataFrame

# Filter coefficients
coefficients = np.array([3.69421067777138E-19, 0.00132722289691133, 0.00465025225065795, 0.0117498026081424, 0.023779221956107, 0.0407234511898748, 0.0611569261162345, 0.082403674188359, 0.101082211156672, 0.113897378800498, 0.118459717673086, 0.113897378800498, 0.101082211156672, 0.082403674188359, 0.0611569261162345, 0.0407234511898748, 0.0237792219561071, 0.0117498026081424, 0.00465025225065795, 0.00132722289691133, 3.69421067777138E-19], dtype=float)

# Apply the filter to each '_oxy' and '_deoxy' column
for column in data.columns:
    if '_oxy' in column or '_deoxy' in column:
        # Ensure the column data is in a suitable numeric format
        numeric_column = pd.to_numeric(data[column], errors='coerce')
        # Apply the filter
        data[column] = lfilter(coefficients, 1, numeric_column)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'data' is your main DataFrame

# List of '_oxy' columns
oxy_columns = [col for col in data.columns if '_oxy' in col]

# Create a new DataFrame that includes only 'condition' and the '_oxy' columns
oxy_data = data[['condition'] + oxy_columns]

# Group by 'condition' and calculate the mean for each group
grouped_oxy_means = oxy_data.groupby('condition')[oxy_columns].mean()

# Transpose the DataFrame to get conditions as columns and optodes as rows
transposed_means = grouped_oxy_means.T

# Plotting
ax = transposed_means.plot(kind='bar', figsize=(15, 8))

plt.xlabel('Optodes')
plt.ylabel('Average Oxy Data')
plt.title('Average Oxy Data by Condition and Optode')
plt.xticks(rotation=45, ha="right")
plt.legend(title='Condition')

plt.tight_layout()
plt.show()

In [ ]:
# Select columns for plotting
oxy_columns = [col for col in data.columns if '_oxy' in col]
plot_data = data[['time'] + oxy_columns]

# Plotting
plt.figure(figsize=(12, 8))
for col in oxy_columns:
    plt.plot(plot_data['time'], plot_data[col], label=col)

plt.xlabel('Time')
plt.ylabel('Oxy Data')
plt.title('Oxy Data Over Time')
plt.legend()
plt.show()

# Analysis - Multiple

In [1]:
import pandas as pd
import numpy as np
from scipy.signal import lfilter

# List of subjects (oxy & mrk suffixes added automatically)
subjects = [
    's1',
    's2',
    's3',
    's4',
    's5',
    's6',
    # 'acb_1_123',
    # 'acb_2_123',
]

paths = [(f'{subject}.oxy', f'{subject}.mrk', i+1) for i, subject in enumerate(subjects)]
gitpath = "https://raw.githubusercontent.com/beallac/iitsec2023/main/"
paths = [(f'{gitpath}{path[0]}', f'{gitpath}{path[1]}', path[2]) for path in paths]


def process_subject_data(oxy_file, marker_file, subject_number):
    # Generate column names
    column_names = ['time'] + [f'opt_{i}_{x}' for i in range(1, 19) for x in ['deoxy', 'oxy']] + ['counter']

    # Load the data
    data = pd.read_csv(oxy_file, delimiter='\t', header=None, skiprows=8, names=column_names)
    marker = pd.read_csv(marker_file, delimiter='\t', header=None, skiprows=6, names=['time_stamp', 'marker_code', 'unused_code'])

    # Process marker data
    # Process marker data to find start and stop times for each condition
    # Assuming consecutive rows with the same marker indicate a continuous condition
    conditions = {}
    for index, row in marker.iterrows():
        marker = row['marker_code']
        if marker in conditions:
            # Assuming markers always come in pairs, we append the stop time
            conditions[marker].append(row['time_stamp'])
        else:
            # Start of a new marker condition
            conditions[marker] = [row['time_stamp']]

    # Ensure each marker has a pair, if not, assign 'inf' as the stop time
    for marker, times in conditions.items():
        if len(times) % 2 != 0:
            times.append(float('inf'))

    # Add a condition index to the main data frame
    def get_condition(time):
        for marker, times in conditions.items():
            for i in range(0, len(times), 2):
                if times[i] <= time < times[i + 1]:
                    return marker
        return None

    # Add condition index
    data.insert(1, 'condition', data['time'].apply(get_condition))

    # Filter the data
    # Filter coefficients
    coefficients = np.array([3.69421067777138E-19, 0.00132722289691133, 0.00465025225065795, 0.0117498026081424, 0.023779221956107, 0.0407234511898748, 0.0611569261162345, 0.082403674188359, 0.101082211156672, 0.113897378800498, 0.118459717673086, 0.113897378800498, 0.101082211156672, 0.082403674188359, 0.0611569261162345, 0.0407234511898748, 0.0237792219561071, 0.0117498026081424, 0.00465025225065795, 0.00132722289691133, 3.69421067777138E-19], dtype=float)

    # Apply the filter to each '_oxy' and '_deoxy' column
    for column in data.columns:
        if '_oxy' in column or '_deoxy' in column:
            # Ensure the column data is in a suitable numeric format
            numeric_column = pd.to_numeric(data[column], errors='coerce')
            # Apply the filter
            data[column] = lfilter(coefficients, 1, numeric_column)

    # Add a column for subject number
    data.insert(1, 'subject', subject_number)

    return data

# Process data for each subject and concatenate into a single DataFrame
all_data = pd.concat([process_subject_data(oxy_file, marker_file, subject_number) for oxy_file, marker_file, subject_number in paths])

# Now all_data contains the processed data for all subjects


In [ ]:
import matplotlib.pyplot as plt

# Assuming 'data' is your main DataFrame

# List of '_oxy' columns
oxy_columns = [col for col in all_data.columns if '_oxy' in col]

# Create a new DataFrame that includes only 'condition' and the '_oxy' columns
oxy_data = all_data[['condition'] + oxy_columns]

# Group by 'condition' and calculate the mean for each group
grouped_oxy_means = oxy_data.groupby('condition')[oxy_columns].mean()

# Transpose the DataFrame to get conditions as columns and optodes as rows
transposed_means = grouped_oxy_means.T

# Plotting
ax = transposed_means.plot(kind='bar', figsize=(15, 8))

plt.xlabel('Optodes')
plt.ylabel('Average Oxy Data')
plt.title('Average Oxy Data by Condition and Optode')
plt.xticks(rotation=45, ha="right")
plt.legend(title='Condition')

plt.tight_layout()
plt.show()